# MIRI PSF 光度测量与 Photutils

**作者**: Ori Fox<br>

**提交时间**: 2023年11月<br>

**更新时间**: 2023年11月<br>

**使用案例**: 使用 [Photutils](https://photutils.readthedocs.io/en/stable/) 进行 PSF 光度测量。此处的目的是展示在多种使用案例中使用 Photutils 的工作流程和运行时间。

一般来说，来自空间望远镜的数据的 PSF 光度测量在预拼接数据上进行时最为准确。拼接过程会改变固有的 PSF，由于重采样和不同探测器位置及旋转下的 PSF 混合，导致其模糊。此外，针对拼接数据的准确理论 PSF 模型（例如，来自 [WebbPSF](https://webbpsf.readthedocs.io/en/latest/)）并不可用。虽然可以为拼接数据构建经验 PSF（例如，使用 Photutils 的 [ePSFBuilder](https://photutils.readthedocs.io/en/latest/epsf.html)），但结果通常不如在预拼接数据上进行 PSF 光度测量准确。

**注意**: 还有一个伴随的笔记本，展示如何使用一个名为 space_phot 的新软件程序在 Level 2 和 Level 3 数据上执行 PSF 光度测量。<br>

**数据**: MIRI 数据 PID 1028（校准程序；单星访问 006 A5V 矮星 2MASSJ17430448+6655015）和 MIRI 数据 PID 1171（LMC；多星）。<br>

**工具**: photutils, webbpsf, jwst <br>

**跨仪器**: MIRI<br>

**文档**: 本笔记本是 STScI 更大 [后处理数据分析工具生态系统](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis) 的一部分，可以直接从 [JDAT Notebook Github 目录](https://github.com/spacetelescope/jdat_notebooks) [下载](https://github.com/spacetelescope/dat_pyinthesky/tree/main/jdat_notebooks/MRS_Mstar_analysis)。<br>

## 目录

1. [引言](#intro)<br>

    1.1 [Python 导入](#imports)<br>

    1.2 [设置 WebbPSF 和 Synphot](#setup)<br>

2. [下载 JWST MIRI 数据](#data)<br>

3. [单一亮度对象](#bso)<br>

    3.1 [单个 Level 2 文件](#bso2)<br>

    3.2 [使用 WebbPSF 生成 MIRI F770W 的经验 PSF 网格](#bso3)<br>

    3.3 [PSF 光度测量](#bso4)<br>

4. [微弱/上限，单一对象](#fso)<br>

    4.1 [多个 Level 2 文件](#fso2)<br>

5. [恒星场 (LMC)](#lmc)<br>

    5.1 [多个恒星，单个 Level 2 文件](#lmc2)<br>

    5.2 [使用 WebbPSF 生成 MIRI F560W 的经验 PSF 网格](#grid2)<br>

    5.3 [PSF 光度测量](#lmc3)<br>

# 1. 引言 <a class="anchor" id="intro"></a>

**目标**：<br>

使用来自WebbPSF的经验PSF模型网格，通过[Photutils PSF Photometry工具](https://photutils.readthedocs.io/en/latest/psf.html)对JWST MIRI图像进行PSF光度测量。

该笔记本展示了如何：<br>

* 从WebbPSF生成[经验PSF模型网格](https://webbpsf.readthedocs.io/en/latest/psf_grids.html)<br>

* 使用[PSFPhotometry类](https://photutils.readthedocs.io/en/latest/api/photutils.psf.PSFPhotometry.html#photutils.psf.PSFPhotometry)对图像进行PSF光度测量<br>

**数据**：<br>

MIRI数据PID 1028（校准程序），F770W <br>

MIRI数据PID 1171（大麦哲伦云），F560W/F770W

## 1.1 Python 导入 <a class="anchor" id="imports"></a>

In [ ]:
import glob  # 导入glob模块，用于文件路径操作

import os  # 导入os模块，用于操作系统功能

import shutil  # 导入shutil模块，用于文件和目录的高阶操作

import tarfile  # 导入tarfile模块，用于处理tar归档文件

from pandas import DataFrame  # 从pandas库导入DataFrame类，用于数据处理

import matplotlib.pyplot as plt  # 导入matplotlib.pyplot模块，用于绘图

import numpy as np  # 导入numpy库，用于数值计算

import webbpsf  # 导入webbpsf库，用于JWST光学模拟

from urllib.parse import urlparse  # 从urllib.parse模块导入urlparse函数，用于解析URL

import requests  # 导入requests库，用于发送HTTP请求

import astropy.units as u  # 导入astropy.units模块，用于单位处理

from astropy.coordinates import SkyCoord  # 从astropy.coordinates模块导入SkyCoord类，用于天文坐标处理

from astropy.io import fits  # 从astropy.io模块导入fits，用于处理FITS文件格式

from astropy.nddata import extract_array  # 从astropy.nddata模块导入extract_array函数，用于数组提取

from astropy.table import Table  # 从astropy.table模块导入Table类，用于表格数据处理

from astropy.visualization import simple_norm  # 从astropy.visualization模块导入simple_norm函数，用于数据可视化

from astroquery.mast import Observations  # 从astroquery.mast模块导入Observations类，用于查询MAST数据

from jwst.datamodels import ImageModel  # 从jwst.datamodels模块导入ImageModel类，用于JWST图像数据模型

from photutils.aperture import CircularAperture  # 从photutils.aperture模块导入CircularAperture类，用于圆形光圈

from photutils.background import LocalBackground, MADStdBackgroundRMS, MMMBackground  # 从photutils.background模块导入背景处理类

from photutils.detection import DAOStarFinder  # 从photutils.detection模块导入DAOStarFinder类，用于星点检测

from photutils.psf import GriddedPSFModel, PSFPhotometry  # 从photutils.psf模块导入PSF模型和光度测量类

## 1.2 下载并设置WebbPSF和Synphot所需的数据 <a class="anchor" id="setup"></a>

In [ ]:
import os  # 导入操作系统模块
import requests  # 导入请求模块
import tarfile  # 导入tarfile模块用于处理tar文件
from urllib.parse import urlparse  # 导入urlparse用于解析URL

# 设置环境变量
os.environ["WEBBPSF_PATH"] = "./webbpsf-data/webbpsf-data"  # 设置WEBBPSF数据路径
os.environ["PYSYN_CDBS"] = "./grp/redcat/trds/"  # 设置PYSYN数据路径

# 所需的webbpsf数据
boxlink = 'https://stsci.box.com/shared/static/qxpiaxsjwo15ml6m4pkhtk36c9jgj70k.gz'  # webbpsf数据链接
boxfile = './webbpsf-data/webbpsf-data-LATEST.tar.gz'  # 下载的webbpsf数据文件路径
synphot_url = 'http://ssb.stsci.edu/trds/tarfiles/synphot5.tar.gz'  # synphot数据链接
synphot_file = './synphot5.tar.gz'  # 下载的synphot数据文件路径

webbpsf_folder = './webbpsf-data'  # webbpsf数据文件夹路径
synphot_folder = './grp'  # synphot数据文件夹路径

# 定义下载文件的函数
def download_file(url, dest_path, timeout=60):
    parsed_url = urlparse(url)  # 解析URL

    if parsed_url.scheme not in ["http", "https"]:  # 检查URL协议是否支持
        raise ValueError(f"Unsupported URL scheme: {parsed_url.scheme}")  # 抛出不支持的协议错误

    response = requests.get(url, stream=True, timeout=timeout)  # 发送GET请求
    response.raise_for_status()  # 检查请求是否成功
    with open(dest_path, "wb") as f:  # 以二进制写入模式打开目标文件
        for chunk in response.iter_content(chunk_size=8192):  # 分块读取内容
            f.write(chunk)  # 写入文件

# 收集webbpsf文件
psfExist = os.path.exists(webbpsf_folder)  # 检查webbpsf文件夹是否存在

if not psfExist:  # 如果文件夹不存在
    os.makedirs(webbpsf_folder)  # 创建webbpsf文件夹
    download_file(boxlink, boxfile)  # 下载webbpsf数据
    gzf = tarfile.open(boxfile)  # 打开tar文件
    gzf.extractall(webbpsf_folder)  # 解压到webbpsf文件夹

# 收集synphot文件
synExist = os.path.exists(synphot_folder)  # 检查synphot文件夹是否存在

if not synExist:  # 如果文件夹不存在
    os.makedirs(synphot_folder)  # 创建synphot文件夹
    download_file(synphot_url, synphot_file)  # 下载synphot数据
    gzf = tarfile.open(synphot_file)  # 打开tar文件
    gzf.extractall('./')  # 解压到当前目录

# 2. 下载JWST MIRI数据 <a class="anchor" id="data"></a>

In [ ]:
# 下载提案ID 1028的F770W数据

# 定义源目录和目标目录
source_dir = 'mastDownload/JWST/'  # 源目录路径
destination_dir = 'mast/01028/'     # 目标目录路径

# 检查目标目录是否已存在
if os.path.isdir(destination_dir):
    print(f'Data already downloaded to {os.path.abspath(destination_dir)}')  # 如果已存在，打印信息
else:
    # 查询MAST（太空望远镜档案库）数据库中提案ID为1028且使用F770W滤光片的观测数据
    obs = Observations.query_criteria(proposal_id=1028, filters=['F770W'])

    # 获取与找到的观测相关的产品列表
    plist = Observations.get_product_list(obs)

    # 过滤产品列表，仅包括特定的产品子组
    fplist = Observations.filter_products(plist, productSubGroupDescription=['CAL', 'I2D', 'ASN'])

    # 从MAST数据库下载选定的产品
    Observations.download_products(fplist)

    # 创建目标目录
    os.makedirs(destination_dir)

    # 使用glob查找所有匹配模式的文件
    files_to_copy = glob.glob(os.path.join(source_dir, 'j*/jw01028*'))

    # 将匹配的文件复制到目标目录
    for file_path in files_to_copy:
        shutil.copy(file_path, destination_dir)  # 复制文件

In [ ]:
# 下载提案ID 1171的F550W和F770W数据

# 定义源目录和目标目录
source_dir = 'mastDownload/JWST/'  # 源数据目录
destination_dir = 'mast/01171/'     # 目标数据目录

# 检查目标目录是否已存在
if os.path.isdir(destination_dir):
    print(f'Data already downloaded to {os.path.abspath(destination_dir)}')  # 如果已下载，打印消息
else:
    # 查询MAST（米库尔斯基太空望远镜档案）数据库以获取观察数据
    # 使用提案ID 1171和F550W、F770W滤光片
    obs = Observations.query_criteria(proposal_id=1171, filters=['F560W', 'F770W'])

    # 获取与找到的观察数据相关的产品列表
    plist = Observations.get_product_list(obs)

    # 过滤产品列表，仅包含特定的产品子组
    fplist = Observations.filter_products(plist, productSubGroupDescription=['CAL', 'I2D', 'ASN'])

    # 从MAST数据库下载所选产品
    Observations.download_products(fplist)

    # 创建目标目录
    os.makedirs(destination_dir)

    # 使用glob查找所有匹配模式的文件
    files_to_copy = glob.glob(os.path.join(source_dir, 'j*/jw01171*'))

    # 将匹配的文件复制到目标目录
    for file_path in files_to_copy:
        shutil.copy(file_path, destination_dir)  # 复制文件

# 3. 单一明亮星星 <a class="anchor" id="bso"></a>

本节的目的是说明如何对单个亮星进行点扩散函数（PSF）光度测量。虽然在孤立情况下，光圈光度测量是可行的，但用户可能会发现，在拥挤的场域或复杂的背景中，PSF光度测量更为可取。

## 3.1 单级别2文件 <a class="anchor" id="bso2"></a>

在这个示例中，我们对单个亮源在单个Level 2图像中进行了拟合。对于一组Level 2图像，我们可以单独拟合每个Level 2图像，然后对测得的通量进行平均。

有用的参考文献：<br>

HST文档关于PSF光度测量： https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/psf<br>

使用HSTPHOT进行WFPC2恒星光度测量： https://ui.adsabs.harvard.edu/abs/2000PASP..112.1383D/abstract<br>

Photutils PSF拟合光度测量： https://photutils.readthedocs.io/en/stable/psf.html

In [ ]:
# 获取二级数据文件名

path = "./mast/01028/"  # 设置数据文件所在路径

level2 = sorted(glob.glob(os.path.join(path, '*cal.fits')))  # 查找并排序所有以'cal.fits'结尾的文件

level2  # 输出找到的二级数据文件名

In [ ]:
# 显示第一个二级图像

data = fits.getdata(level2[0])  # 从level2列表中获取第一个图像数据

norm = simple_norm(data, 'sqrt', percent=99)  # 使用平方根归一化数据，设置99%的百分位数

fig, ax = plt.subplots(figsize=(20, 12))  # 创建一个20x12英寸的图形和坐标轴

im = ax.imshow(data, origin='lower', norm=norm, cmap='gray')  # 显示图像，设置原点在下方，应用归一化和灰度色图

clb = plt.colorbar(im, label='MJy/sr')  # 添加颜色条，并标注单位为MJy/sr

ax.set_xlabel('Pixels')  # 设置x轴标签为“Pixels”

ax.set_ylabel('Pixels')  # 设置y轴标签为“Pixels”

In [ ]:
# 将所有DQ标记的像素值更改为NaN。

# 在这里，我们将覆盖原始的CAL文件。

# JWST DQ标记定义参考：https://jwst-pipeline.readthedocs.io/en/latest/jwst/references_general/references_general.html

# 在这种情况下，我们选择所有DQ值大于10的像素，但用户可以根据自己的需要选择其他值。

filename = level2[0]  # 获取第一个level2文件名

with ImageModel(filename) as model:  # 使用ImageModel打开文件

    model.data[model.dq >= 10] = np.nan  # 将DQ值大于等于10的像素设置为NaN

    model.save(filename)  # 保存修改后的文件

In [ ]:
# 重新显示图像

data = fits.getdata(level2[0])  # 从level2中获取数据

norm = simple_norm(data, 'sqrt', percent=99)  # 归一化数据，使用平方根缩放，99百分位数

fig, ax = plt.subplots(figsize=(20, 12))  # 创建一个20x12英寸的图形和坐标轴

im = ax.imshow(data, origin='lower', norm=norm, cmap='gray')  # 显示图像，设置原点在下方，应用归一化和灰度色图

clb = plt.colorbar(im, label='MJy/sr')  # 添加颜色条，标签为'MJy/sr'

ax.set_xlabel('Pixels')  # 设置x轴标签为'Pixels'

ax.set_ylabel('Pixels')  # 设置y轴标签为'Pixels'

In [ ]:
# 放大以查看源。在这种情况下，我们的源来自MIRI程序ID #1028，一个校准程序。

# 我们使用访问006，目标是A5V矮星2MASSJ17430448+6655015

# 参考链接: http://simbad.cds.unistra.fr/simbad/sim-basic?Ident=2MASSJ17430448%2B6655015&submit=SIMBAD+search

source_location = SkyCoord('17:43:04.4879', '+66:55:01.837', unit=(u.hourangle, u.deg))  # 定义源的位置，使用天球坐标

with ImageModel(filename) as model:  # 使用图像模型打开文件

    x, y = model.meta.wcs.world_to_pixel(source_location)  # 将天球坐标转换为像素坐标

cutout = extract_array(data, (21, 21), (y, x))  # 从数据中提取一个21x21的切片

fig, ax = plt.subplots(figsize=(8, 8))  # 创建一个8x8英寸的图形和坐标轴

norm2 = simple_norm(cutout, 'log', percent=99)  # 使用对数归一化方法对切片进行归一化，99%分位数

im = ax.imshow(cutout, origin='lower', norm=norm2, cmap='gray')  # 显示切片图像，设置原点在下方，使用灰度色图

clb = plt.colorbar(im, label='MJy/sr', shrink=0.8)  # 添加颜色条，标签为'MJy/sr'，缩小到80%

ax.set_title('PID1028, Obs006')  # 设置图形标题

ax.set_xlabel('Pixels')  # 设置x轴标签为'像素'

ax.set_ylabel('Pixels')  # 设置y轴标签为'像素'

## 3.2 使用WebbPSF生成MIRI F770W的经验点扩散函数（PSF）网格 <a class="anchor" id="bso3"></a>

现在我们使用WebbPSF生成MIRI F770W的经验ePSF模型网格。

输出将是一个Photutils [GriddedPSFModel](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GriddedPSFModel.html#photutils.psf.GriddedPSFModel)，包含一个2x2的网格，每个网格都是依赖于探测器位置的经验PSF，且每个PSF的过采样因子为4。请注意，我们将PSF网格保存为一个FITS文件（通过 `save=True`），文件名为 `miri_mirim_f770w_fovp101_samp4_npsf4.fits`。为了节省未来运行的时间，我们将此FITS文件直接加载到一个 `GriddedPSFModel` 对象中：

In [ ]:
psfgrid_filename = 'miri_mirim_f770w_fovp101_samp4_npsf4.fits'  # 定义PSF网格文件名

if not os.path.exists(psfgrid_filename):  # 检查文件是否存在

    miri = webbpsf.MIRI()  # 创建MIRI望远镜的实例

    miri.filter = 'F770W'  # 设置滤光片为F770W

    psf_model = miri.psf_grid(num_psfs=4, all_detectors=True, verbose=True, save=True)  # 生成PSF网格模型并保存

else:

    psf_model = GriddedPSFModel.read(psfgrid_filename)  # 从文件中读取PSF网格模型

psf_model  # 输出PSF模型

In [ ]:
# 显示点扩散函数（PSF）网格

psf_model.plot_grid()  # 调用psf_model对象的plot_grid方法以绘制PSF网格

In [ ]:
# 显示来自平均电子点扩散函数（ePSF）的点扩散函数（PSF）网格差异

psf_model.plot_grid(deltas=True)  # 调用psf_model对象的plot_grid方法，设置deltas参数为True以显示差异

## 3.3 PSF 光度测量 <a class="anchor" id="bso4"></a>

现在让我们使用我们的网格化点扩散函数（PSF）模型来进行PSF光度测量。

In [ ]:
# 加载数据并将单位从 MJy/sr 转换为 uJy

with ImageModel(filename) as model:  # 使用 ImageModel 打开文件并创建模型

    unit = u.Unit(model.meta.bunit_data)  # 获取数据的单位

    data = model.data << unit  # 将模型数据转换为指定单位

    error = model.err << unit  # 将模型误差转换为指定单位

    # 使用像素面积图，因为二级数据中存在几何失真

    pixel_area = model.area * model.meta.photometry.pixelarea_steradians * u.sr  # 计算像素面积

    data *= pixel_area  # 将数据乘以像素面积

    error *= pixel_area  # 将误差乘以像素面积

    

    data = data.to(u.uJy)  # 将数据转换为 uJy

    error = error.to(u.uJy)  # 将误差转换为 uJy

data.unit, error.unit  # 返回数据和误差的单位

要对单个源进行光度测量，我们可以输入一个包含其 (x, y) 位置的表格。

In [ ]:
# 初始化一个空的表格
init_params = Table()

# 在表格中添加一列'x'，并将其值设置为[x]
init_params['x'] = [x]

# 在表格中添加一列'y'，并将其值设置为[y]
init_params['y'] = [y]

# 返回初始化后的表格
init_params

In [ ]:
# 我们关闭查找器，因为我们输入了源位置

fit_shape = 5  # 设置拟合形状的大小

# 创建局部背景估计器，使用MMMBackground作为背景估计方法
localbkg_estimator = LocalBackground(5, 10, bkg_estimator=MMMBackground())

# 创建PSF光度测量对象，使用指定的PSF模型和参数
psfphot = PSFPhotometry(psf_model, fit_shape, finder=None, aperture_radius=fit_shape, 

                        localbkg_estimator=localbkg_estimator, progress_bar=True)  # 进度条设置为True

# 使用数据和误差进行光度测量，初始化参数为init_params
phot = psfphot(data, error=error, init_params=init_params)

phot  # 返回光度测量结果

In [ ]:
# 将光度从微杰（uJy）转换为AB星等

flux = phot['flux_fit']  # 获取拟合光度

flux_err = phot['flux_err']  # 获取光度误差

mag = phot['flux_fit'].to(u.ABmag)  # 将光度转换为AB星等

magerr = 2.5 * np.log10(1.0 + (flux_err / flux))  # 计算星等误差

magerr = magerr.value * u.ABmag  # 将误差值转换为AB星等单位

mag, magerr  # 返回星等和星等误差

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))  # 创建一个包含3列的子图，图像大小为12x4英寸

shape = (21, 21)  # 定义切割区域的形状为21x21

cutout1 = extract_array(data.value, shape, (y, x))  # 从数据中提取指定位置的切割区域

norm = simple_norm(cutout1, 'log', percent=98)  # 使用对数归一化方法对切割区域进行归一化，取前98%的数据

im1 = ax[0].imshow(cutout1, origin='lower', norm=norm)  # 在第一个子图中显示切割区域图像

ax[0].set_title(r'Data ($\mu$Jy)')  # 设置第一个子图的标题

plt.colorbar(im1, shrink=0.7)  # 为第一个子图添加颜色条，并缩小到原来的70%

model = psfphot.make_model_image(data.shape, shape)  # 创建与数据形状相同的PSF模型图像

cutout2 = extract_array(model, shape, (y, x))  # 从模型中提取指定位置的切割区域

im2 = ax[1].imshow(cutout2, origin='lower', norm=norm)  # 在第二个子图中显示模型图像

ax[1].set_title('Fit PSF Model')  # 设置第二个子图的标题

plt.colorbar(im2, shrink=0.7)  # 为第二个子图添加颜色条，并缩小到原来的70%

resid = psfphot.make_residual_image(data.value, shape)  # 创建残差图像

cutout3 = extract_array(resid, shape, (y, x))  # 从残差中提取指定位置的切割区域

norm3 = simple_norm(cutout3, 'sqrt', percent=99)  # 使用平方根归一化方法对残差进行归一化，取前99%的数据

im3 = ax[2].imshow(cutout3, origin='lower', norm=norm3)  # 在第三个子图中显示残差图像

ax[2].set_title('Residual')  # 设置第三个子图的标题

plt.colorbar(im3, shrink=0.7)  # 为第三个子图添加颜色条，并缩小到原来的70%

# 4. 微弱/上限，单个天体 <a class="anchor" id="fso"></a>

本节的目的是说明如何在固定的 (x, y) 位置使用强制点扩散函数（PSF）光度法计算天空空白区域的上限。

我们将使用与第 3 节相同的数据。

In [ ]:
# 加载数据并将单位从 MJy/sr 转换为 uJy

with ImageModel(filename) as model:  # 使用 ImageModel 打开文件
    unit = u.Unit(model.meta.bunit_data)  # 获取数据单位

    data = model.data << unit  # 将数据转换为指定单位
    error = model.err << unit  # 将误差转换为指定单位

    pixel_area = model.meta.photometry.pixelarea_steradians * u.sr  # 获取像素面积并转换为立体角单位

    data *= pixel_area  # 将数据乘以像素面积
    error *= pixel_area  # 将误差乘以像素面积

    data = data.to(u.uJy)  # 将数据转换为微焦耳单位
    error = error.to(u.uJy)  # 将误差转换为微焦耳单位

# 定义源位置的天球坐标
source_location = SkyCoord('17:43:00.0332', '+66:54:42.677', unit=(u.hourangle, u.deg))

with ImageModel(filename) as model:  # 再次使用 ImageModel 打开文件
    x, y = model.meta.wcs.world_to_pixel(source_location)  # 将天球坐标转换为像素坐标

# 从数据中提取 21x21 的切片
cutout = extract_array(data.value, (21, 21), (y, x))  # 提取以源位置为中心的切片

fig, ax = plt.subplots()  # 创建图形和坐标轴
norm = simple_norm(cutout, 'sqrt', percent=95)  # 计算归一化参数

im = ax.imshow(cutout, origin='lower', norm=norm, cmap='gray')  # 显示切片图像
clb = plt.colorbar(im, label=r'$\mu$Jy')  # 添加颜色条并标注单位

ax.set_title('PID1028, Obs006')  # 设置图像标题

ax.set_xlabel('Pixels')  # 设置 x 轴标签
ax.set_ylabel('Pixels')  # 设置 y 轴标签

In [ ]:
# 为了执行强制光度测量，我们设置源位置 (x, y)

# 并且我们固定PSF模型的位置，以便在拟合中不发生变化

# （仅拟合光通量）

init_params = Table()  # 创建一个表格来存储初始参数

init_params['x'] = [x]  # 将源的x坐标添加到表格中

init_params['y'] = [y]  # 将源的y坐标添加到表格中

# 这需要 photutils 1.11.0 版本

psf_model_forced = psf_model.copy()  # 复制PSF模型以进行强制光度测量

psf_model_forced.x_0.fixed = True  # 固定PSF模型的x位置

psf_model_forced.y_0.fixed = True  # 固定PSF模型的y位置

psf_model_forced.fixed  # 确保模型的固定状态

In [ ]:
fit_shape = 5  # 设置拟合形状的大小为5

# 创建局部背景估计器，使用MMMBackground作为背景估计方法
localbkg_estimator = LocalBackground(5, 10, bkg_estimator=MMMBackground())

# 初始化PSF光度测量对象，设置相关参数
psfphot = PSFPhotometry(psf_model_forced, fit_shape, finder=None, aperture_radius=fit_shape, 
                        localbkg_estimator=localbkg_estimator, progress_bar=True)

# 使用数据和误差进行光度测量，初始化参数为init_params
phot = psfphot(data, error=error, init_params=init_params)

phot  # 输出光度测量结果

In [ ]:
# 计算上限，乘以所需的sigma值

sigma = 3.0  # 设置sigma值为3.0

limit = sigma * phot['flux_err']  # 计算上限，乘以光通量误差

limit.to(u.ABmag)  # 将结果转换为AB星等

## 注意：您可以通过Level 3组合数据产品深入挖掘数据

# 5. 恒星场 (LMC) <a class="anchor" id="lmc"></a>

在这种情况下，我们将执行与第3节相同的步骤，但针对多个恒星。目的是展示在大量恒星上使用Photutils的工作流程和运行时间。

## 5.1 多重星，单个 Level 2 文件 <a class="anchor" id="lmc2"></a>

In [ ]:
# 查找Level 3文件中的恒星

import os  # 导入os模块以处理文件路径

path = './mast/01171/'  # 设置文件路径

level3 = os.path.join(path, 'jw01171-o004_t001_miri_f560w_i2d.fits')  # 组合路径和文件名，生成完整文件路径

level3  # 输出Level 3文件的完整路径

In [ ]:
# 获取背景的粗略估计（还有更好的背景减法方法）

bkgrms = MADStdBackgroundRMS()  # 创建一个背景RMS估计器

mmm_bkg = MMMBackground()  # 创建一个MMM背景估计器

with ImageModel(level3) as model:  # 使用ImageModel上下文管理器打开level3数据

    wcs_l3 = model.meta.wcs  # 获取WCS信息

    std = bkgrms(model.data)  # 计算数据的背景RMS标准差

    bkg = mmm_bkg(model.data)  # 计算数据的MMM背景

    data_bkgsub = model.data.copy()  # 复制原始数据以进行背景减法

    data_bkgsub -= bkg  # 从数据中减去背景

# 查找恒星

# F560W FWHM = 1.882 像素

fwhm_psf = 1.882  # 设置点扩散函数的全宽半最大值

threshold = 5.0  # 设置阈值

daofind = DAOStarFinder(threshold=threshold * std, fwhm=fwhm_psf, exclude_border=True, min_separation=10)  # 创建DAO星点查找器

found_stars = daofind(data_bkgsub)  # 在减去背景的数据中查找恒星

In [ ]:
# 打印找到的星星信息，最多显示10行
found_stars.pprint_all(max_lines=10)

In [ ]:
# 绘制找到的恒星

# 使用平方根归一化方法对背景减去的数据进行归一化，99%为最大值
norm = simple_norm(data_bkgsub, 'sqrt', percent=99)

# 创建一个10x10英寸的图形和坐标轴
fig, ax = plt.subplots(figsize=(10, 10))

# 在坐标轴上显示背景减去的数据，原点位于左下角，使用归一化
ax.imshow(data_bkgsub, origin='lower', norm=norm)

# 将找到的恒星的中心坐标打包为元组
xypos = zip(found_stars['xcentroid'], found_stars['ycentroid'])

# 创建一个半径为10的圆形光圈，表示恒星的位置
aper = CircularAperture(xypos, r=10)

# 在坐标轴上绘制圆形光圈，颜色为红色
aper.plot(ax, color='red')

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(10, 8))  # 创建一个包含2行的子图，图形大小为10x8英寸

ax[0].scatter(found_stars['mag'], found_stars['sharpness'], s=10, color='k')  # 在第一个子图中绘制星星的亮度与锐度的散点图
ax[0].set_xlabel('mag')  # 设置x轴标签为'mag'
ax[0].set_ylabel('sharpness')  # 设置y轴标签为'sharpness'

ax[1].scatter(found_stars['mag'], found_stars['roundness2'], s=10, color='k')  # 在第二个子图中绘制星星的亮度与圆度的散点图
ax[1].set_xlabel('mag')  # 设置x轴标签为'mag'
ax[1].set_ylabel('roundness')  # 设置y轴标签为'roundness'

mag0 = -3.0  # 定义亮度的下界
mag1 = -5.0  # 定义亮度的上界
for ax_ in ax:  # 遍历所有子图
    ax_.axvline(mag0, color='red', linestyle='dashed')  # 在每个子图中绘制亮度下界的虚线
    ax_.axvline(mag1, color='red', linestyle='dashed')  # 在每个子图中绘制亮度上界的虚线

sh0 = 0.40  # 定义锐度的下界
sh1 = 0.82  # 定义锐度的上界
ax[0].axhline(sh0, color='red', linestyle='dashed')  # 在第一个子图中绘制锐度下界的虚线
ax[0].axhline(sh1, color='red', linestyle='dashed')  # 在第一个子图中绘制锐度上界的虚线

rnd0 = -0.40  # 定义圆度的下界
rnd1 = 0.40  # 定义圆度的上界
ax[1].axhline(rnd0, color='red', linestyle='dashed')  # 在第二个子图中绘制圆度下界的虚线
ax[1].axhline(rnd1, color='red', linestyle='dashed')  # 在第二个子图中绘制圆度上界的虚线

In [ ]:
# 创建一个布尔掩码，用于筛选符合条件的星体
mask = ((found_stars['mag'] < mag0) &  # 星体的亮度小于mag0
        (found_stars['mag'] > mag1) &  # 星体的亮度大于mag1
        (found_stars['roundness2'] > rnd0) &  # 星体的圆度大于rnd0
        (found_stars['roundness2'] < rnd1) &  # 星体的圆度小于rnd1
        (found_stars['sharpness'] > sh0) &  # 星体的清晰度大于sh0
        (found_stars['sharpness'] < sh1) &  # 星体的清晰度小于sh1
        (found_stars['xcentroid'] > 100) &  # 星体的x坐标大于100
        (found_stars['xcentroid'] < 700) &  # 星体的x坐标小于700
        (found_stars['ycentroid'] > 100) &  # 星体的y坐标大于100
        (found_stars['ycentroid'] < 700))   # 星体的y坐标小于700

# 根据掩码筛选出符合条件的星体
found_stars_sel = found_stars[mask]

# 输出原始找到的星体数量
print('Number of stars found originally:', len(found_stars))

# 输出最终筛选后的星体数量
print('Number of stars in final selection:', len(found_stars_sel))

In [ ]:
# 绘制选定的恒星

# 使用平方根归一化方法对背景减去的数据进行归一化，99%为阈值
norm = simple_norm(data_bkgsub, 'sqrt', percent=99)

# 创建一个10x10英寸的图形和坐标轴
fig, ax = plt.subplots(figsize=(10, 10))

# 在坐标轴上显示背景减去的数据，设置原点在下方，并应用归一化
ax.imshow(data_bkgsub, origin='lower', norm=norm)

# 获取选定恒星的中心坐标
xypos = zip(found_stars_sel['xcentroid'], found_stars_sel['ycentroid'])

# 创建一个半径为10的圆形光圈，表示恒星的位置
aper = CircularAperture(xypos, r=10)

# 在坐标轴上绘制圆形光圈，颜色为红色
aper.plot(ax, color='red')

## 5.2 使用WebbPSF生成MIRI F560W的经验点扩散函数（PSF）网格 <a class="anchor" id="grid2"></a>

In [ ]:
psfgrid_filename = 'miri_mirim_f560w_fovp101_samp4_npsf4.fits'  # 定义PSF网格文件名

if not os.path.exists(psfgrid_filename):  # 检查文件是否存在

    miri = webbpsf.MIRI()  # 创建MIRI望远镜的实例

    miri.filter = 'F560W'  # 设置使用的滤光片为F560W

    # 生成PSF模型，指定生成4个PSF，适用于所有探测器，启用详细输出，并保存结果
    psf_model = miri.psf_grid(num_psfs=4, all_detectors=True, verbose=True, save=True)

else:  # 如果文件已存在

    # 从现有的PSF网格文件中读取PSF模型
    psf_model = GriddedPSFModel.read(psfgrid_filename)

psf_model  # 输出PSF模型

In [ ]:
# 调用psf_model对象的plot_grid方法以绘制点扩散函数（PSF）网格
psf_model.plot_grid()

In [ ]:
# 获取二级图像

# 在这里，我们将使用PID 1171文件

path = "./mast/01171/"  # 设置文件路径

level2 = sorted(glob.glob(os.path.join(path, 'jw01171004*cal.fits')))  # 获取所有符合条件的文件并排序

filename = level2[0]  # 选择第一个文件

print(filename)  # 打印文件名

# 加载数据并将单位从MJy/sr转换为uJy

with ImageModel(filename) as model:  # 使用ImageModel上下文管理器加载文件

    unit = u.Unit(model.meta.bunit_data)  # 获取数据单位

    model.data[model.dq >= 10] = np.nan  # 将数据质量标志大于等于10的像素设置为NaN

    data = model.data << unit  # 将数据加载并应用单位

    error = model.err << unit  # 将误差加载并应用单位

    

    pixel_area = model.meta.photometry.pixelarea_steradians * u.sr  # 获取像素面积并转换为立体角单位

    data *= pixel_area  # 将数据乘以像素面积

    error *= pixel_area  # 将误差乘以像素面积

    

    data = data.to(u.uJy)  # 将数据单位转换为微杰斯基（uJy）

    error = error.to(u.uJy)  # 将误差单位转换为微杰斯基（uJy）

    wcs = model.meta.wcs  # 获取世界坐标系统信息

data.unit, error.unit  # 返回数据和误差的单位

## 5.3 PSF 光度测量 <a class="anchor" id="lmc3"></a>

In [ ]:
# 将（x，y）位置从三级图像转换为二级图像

xc = found_stars_sel['xcentroid']  # 获取选定星星的x坐标

yc = found_stars_sel['ycentroid']  # 获取选定星星的y坐标

sc = wcs_l3.pixel_to_world(xc, yc)  # 将像素坐标转换为世界坐标

x, y = wcs.world_to_pixel(sc)  # 将世界坐标转换回像素坐标

init_params = Table()  # 创建一个新的表格来存储初始化参数

init_params['x'] = x  # 将x坐标添加到表格中

init_params['y'] = y  # 将y坐标添加到表格中

# 我们需要移除二级数据中被遮罩区域内的星星

mask = x > 400  # 创建一个掩码，选择x坐标大于400的星星

init_params = init_params[mask]  # 根据掩码过滤初始化参数表

In [ ]:
看起来您提到的“mask”可能是指在天文学数据处理中使用的掩膜（mask）。不过，您没有提供具体的代码或上下文。为了帮助您，我将提供一个示例代码，展示如何在JWST数据处理中使用掩膜，并添加中文注释。

import numpy as np  # 导入NumPy库，用于数值计算
import matplotlib.pyplot as plt  # 导入Matplotlib库，用于绘图
from astropy.io import fits  # 导入Astropy库，用于处理FITS文件

# 读取JWST数据的FITS文件
def read_fits_file(file_path):
    """读取FITS文件并返回数据和头信息"""
    with fits.open(file_path) as hdul:  # 打开FITS文件
        data = hdul[1].data  # 获取数据部分
        header = hdul[1].header  # 获取头信息
    return data, header  # 返回数据和头信息

# 创建掩膜函数
def create_mask(data, threshold):
    """根据阈值创建掩膜"""
    mask = data < threshold  # 创建掩膜，数据小于阈值的部分为True
    return mask  # 返回掩膜

# 应用掩膜函数
def apply_mask(data, mask):
    """应用掩膜到数据"""
    masked_data = np.where(mask, np.nan, data)  # 将掩膜为True的部分替换为NaN
    return masked_data  # 返回掩膜后的数据

# 主程序
if __name__ == "__main__":
    file_path = 'jwst_data.fits'  # 指定FITS文件路径
    threshold = 1000  # 设置阈值

    data, header = read_fits_file(file_path)  # 读取FITS文件
    mask = create_mask(data, threshold)  # 创建掩膜
    masked_data = apply_mask(data, mask)  # 应用掩膜

    # 绘制原始数据和掩膜后的数据
    plt.figure(figsize=(12, 6))  # 设置绘图窗口大小
    plt.subplot(1, 2, 1)  # 创建子图1
    plt.imshow(data, cmap='gray', origin='lower')  # 显示原始数据
    plt.title('Original Data')  # 设置标题
    plt.colorbar()  # 添加颜色条

    plt.subplot(1, 2, 2)  # 创建子图2
    plt.imshow(masked_data, cmap='gray', origin='lower')  # 显示掩膜后的数据
    plt.title('Masked Data')  # 设置标题
    plt.colorbar()  # 添加颜色条

    plt.tight_layout()  # 调整子图布局
    plt.show()  # 显示绘图

在这个示例中，我添加了中文注释以帮助理解每一行代码的功能。请根据您的具体需求提供更多信息，以便我能更好地帮助您。

In [ ]:
# 绘制选定的恒星

# 使用平方根归一化数据，设置99%的百分位数
norm = simple_norm(data.value, 'sqrt', percent=99)

# 创建一个10x10英寸的图形和坐标轴
fig, ax = plt.subplots(figsize=(10, 10))

# 在坐标轴上显示数据图像，设置原点在下方，并应用归一化
ax.imshow(data.value, origin='lower', norm=norm)

# 获取初始参数中的x和y坐标
xypos = zip(init_params['x'], init_params['y'])

# 创建一个半径为10的圆形光圈
aper = CircularAperture(xypos, r=10)

# 在坐标轴上绘制光圈，颜色为红色
aper.plot(ax, color='red')

In [ ]:
fit_shape = 5  # 设置拟合形状的大小

# 创建局部背景估计器，使用MMMBackground作为背景估计方法
localbkg_estimator = LocalBackground(5, 10, bkg_estimator=MMMBackground())

# 初始化PSF光度测量对象，设置PSF模型、拟合形状、查找器、光圈半径和局部背景估计器
psfphot = PSFPhotometry(psf_model, fit_shape, finder=None, aperture_radius=fit_shape, 
                        localbkg_estimator=localbkg_estimator, progress_bar=True)

# 使用数据和误差进行光度测量，初始化参数
phot = psfphot(data, error=error, init_params=init_params)

phot  # 返回光度测量结果

In [ ]:
# 将光度从微杰克斯（uJy）转换为AB星等

flux = phot['flux_fit']  # 获取拟合光度

flux_err = phot['flux_err']  # 获取光度误差

mag = phot['flux_fit'].to(u.ABmag)  # 将光度转换为AB星等

magerr = 2.5 * np.log10(1.0 + (flux_err / flux))  # 计算星等误差

magerr = magerr.value * u.ABmag  # 将误差值转换为AB星等单位

mag, magerr  # 返回星等和星等误差

In [ ]:
# 写入文件

# 创建一个DataFrame，包含RA、DEC、Magnitude和Magnitude误差
df = DataFrame({"RA": sc.ra.deg[mask],  # RA列，使用掩码筛选的RA角度（度）
                 "DEC": sc.dec.deg[mask],  # DEC列，使用掩码筛选的DEC角度（度）
                 "Mag": mag.value,  # Magnitude列，存储星等值
                 "Mag Err": magerr.value})  # Magnitude Err列，存储星等误差值

# 将DataFrame写入文本文件，不包含索引
df.to_csv('miri_photometry_photutils.txt', index=False)

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="太空望远镜标志" width="200px"/>